In [ ]:
# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Step 2: Install TensorFlow Lite
!pip install tflite-runtime

# Step 3: Import Required Libraries
import numpy as np
import tensorflow.lite as tflite
from PIL import Image

# Step 4: Define Paths (Update with your actual filenames)
MODEL_PATH = "/content/wheat_leaf_disease_model (2).tflite"  # Update your model file name
IMAGE_PATH = "/content/Brown_rust006_aug_523.jpg"  # Update your test image file name

# Step 5: Load TFLite Model
interpreter = tflite.Interpreter(model_path=MODEL_PATH)
interpreter.allocate_tensors()

# Step 6: Get Model Input/Output Details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Step 7: Load and Preprocess Image
def preprocess_image(image_path):
    image = Image.open(image_path).resize((224, 224))  # Resize to model input size
    image = np.array(image, dtype=np.float32) / 255.0  # Normalize (0-1)
    image = np.expand_dims(image, axis=0)  # Convert to (1, 224, 224, 3)
    return image

input_data = preprocess_image(IMAGE_PATH)

# Step 8: Run Inference
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()

# Step 9: Get Prediction
output_data = interpreter.get_tensor(output_details[0]['index'])
predicted_class = np.argmax(output_data)  # Get class index

# Step 10: Define Class Labels
CLASS_LABELS = ["Brown Rust", "Healthy", "Yellow Rust"]  # Adjust if needed

# Step 11: Print Prediction
print(f"Predicted Class: {CLASS_LABELS[predicted_class]}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Predicted Class: Brown Rust


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Conv2D, MaxPooling2D, DepthwiseConv2D, BatchNormalization, LeakyReLU, Concatenate
import os

# Function to create InceptionV3 architecture with Transfer Learning
def inceptionv3_base(input_shape=(224, 224, 3)):
    base_model = tf.keras.applications.InceptionV3(weights=None, include_top=False, input_shape=input_shape)
    x = base_model.output
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = GlobalAveragePooling2D()(x)
    return Model(base_model.input, x)

# Function to create Custom-CNN from scratch
def custom_cnn_base(input_shape=(224, 224, 3)):
    inputs = tf.keras.Input(shape=input_shape)
    x = Conv2D(32, (3, 3), strides=(2, 2), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = DepthwiseConv2D((3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = Conv2D(64, (1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = DepthwiseConv2D((3, 3), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = Conv2D(128, (1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = GlobalAveragePooling2D()(x)
    return Model(inputs, x)

# Build model
inception_base = inceptionv3_base(input_shape=(224, 224, 3))
custom_cnn_base = custom_cnn_base(input_shape=(224, 224, 3))
inputs = tf.keras.Input(shape=(224, 224, 3))
inception_features = inception_base(inputs)
custom_cnn_features = custom_cnn_base(inputs)
x = Concatenate()([inception_features, custom_cnn_features])
x = Dense(1024, activation='relu')(x)
outputs = Dense(3, activation='softmax')(x)
model = Model(inputs, outputs)

# Load weights
weights_path = "/content/hybrid_model_weights.weights (3).h5"  # Update path
model.load_weights(weights_path)
print("✅ Weights loaded successfully!")

# Function to preprocess image
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img) / 255.0  # Normalize
    return np.expand_dims(img_array, axis=0)  # Add batch dimension

# Test the model
def predict(img_path):
    img_array = preprocess_image(img_path)
    preds = model.predict(img_array)
    class_names = ['Healthy', 'Yellow Rust', 'Brown Rust']  # Update class labels if needed
    predicted_class = class_names[np.argmax(preds)]
    print(f"Predicted Class: {predicted_class}")

# Run test
image_path = "/content/Yellow_rust069.jpg"  # Update with your test image path
predict(image_path)


ValueError: Layer count mismatch when loading weights from file. Model expected 4 layers, found 0 saved layers.